In [122]:
import gensim.downloader as api
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import os

In [186]:
word_vectors = api.load("glove-twitter-200") 

[=-------------------------------------------------] 3.2% 24.5/758.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[===-----------------------------------------------] 6.5% 49.3/758.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[================----------------------------------] 32.2% 244.0/758.5MB downloaded

...

[==========================------------------------] 53.4% 405.4/758.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=================================-----------------] 66.4% 503.4/758.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 758.5/758.5MB downloaded


In [187]:
def preprocess(text):
    return [word for word in text.lower().split()]

In [188]:
def document_vector(doc):
    doc = [word for word in doc if word in word_vectors.key_to_index]
    if len(doc) == 0:
        return np.zeros(word_vectors.vector_size)
    return np.mean(word_vectors[doc], axis=0)

In [191]:
df

,sentiment,review,tokens,vector
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[@switchfoot, http://twitpic.com/2y1zl, -, aww...","[0.08728404, 0.16949035, 0.15050696, 0.0520680..."
1,0,is upset that he can't update his Facebook by ...,"[is, upset, that, he, can't, update, his, face...","[0.18049781, 0.16101497, -0.0043811747, 0.0954..."
2,0,@Kenichan I dived many times for the ball. Man...,"[@kenichan, i, dived, many, times, for, the, b...","[0.21760783, 0.16113253, 0.25067896, -0.014643..."
3,0,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[-0.09874002, -0.211591, -0.057362698, 0.17540..."
4,0,"@nationwideclass no, it's not behaving at all....","[@nationwideclass, no,, it's, not, behaving, a...","[0.106674924, 0.371483, -0.034092028, 0.223679..."
...,...,...,...,...
1599995,4,Just woke up. Having no school is the best fee...,"[just, woke, up., having, no, school, is, the,...","[0.17664075, 0.19861221, 0.19899783, 0.0638722..."
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,"[thewdb.com, -, very, cool, to, hear, old, wal...","[0.13821416, -0.05230285, -0.047131293, 0.1906..."
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,"[are, you, ready, for, your, mojo, makeover?, ...","[0.27056298, 0.195423, -0.14965133, -0.0431367..."
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,"[happy, 38th, birthday, to, my, boo, of, alll,...","[0.102873035, 0.28568122, -0.005931119, 0.1250..."


In [190]:
df = df[df['review'].notnull()].copy()
df['tokens'] = df['review'].apply(preprocess)
df['vector'] = df['tokens'].apply(document_vector)

In [192]:
X = np.vstack(df['vector'].values)
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000, class_weight='balanced')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74    159494
           4       0.74      0.74      0.74    160506

    accuracy                           0.74    320000
   macro avg       0.74      0.74      0.74    320000
weighted avg       0.74      0.74      0.74    320000



In [193]:
def predict_sentiment(text):
    tokens = preprocess(text)
    vec = document_vector(tokens).reshape(1,-1)
    return clf.predict(vec)[0]

In [214]:
print(predict_sentiment("I love this product!"))
print(predict_sentiment("This is the worst thing I bought."))

4
0
